## <span>PREDICTIVE ANALYSIS</span>

### **Question**
Which clients are most likely to abandon the telecommunications firm, and why? What characteristics are particularly significant in terms of reducing customer churn? The predictive method that will be utilized is decision trees.

### **Goal**
Organizational stakeholders will benefit from knowing, with some confidence, which consumers are more likely to churn and which are not. As a result of statistical analysis, stakeholders will be able to acquire answers to their questions and make better business decisions. Understanding the churn rate will also help you sell better services to customers and understand previous user experiences.

### **Justification for Decision Tree Method**

The classification and regression tree is made up of a sequence of if/else queries about specific categorical variables that are used to infer labels. Without feature scaling, the classification tree can capture non-linear connections between features and labels. Every branch represents an attribute, and at the conclusion of each node, the decision tree will determine the optimal decision.

### **Assumptions**

At first, the entire training set is regarded as the root. Categorical feature values are desired. If the values are continuous, they must be discretized before the model can be built. On the basis of attribute values, records are dispersed iteratively.

### **Prepared Dataset**

In [22]:
newchurns = pd.read_csv('DecisionTreeClassfication.csv')

In [23]:
newchurns.shape

(10000, 35)

In [24]:
#Predictor Variables will be stored into X. Y will have the values of EncodedChurn. 
x = newchurns.drop('EncodedChurn', axis=1).values
y = newchurns['EncodedChurn'].values

In [25]:
#Import Scikit-learn Libraries
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeRegressor 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import confusion_matrix, classification_report

### **Splitting the data into training and test data sets**

In [26]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.3, stratify= y, random_state = 1)

### **Analysis Technique and Intermediate Calculations**

In [27]:
#Instantiate dt 
dt = DecisionTreeClassifier(max_depth = None, random_state = 1)
#Fit dt to training set 
dt.fit(x_train, y_train)
#predict test set labels 
y_prediction = dt.predict(x_test)
print(y_prediction[0:5])
#accuracy_score 
acc_score = accuracy_score(y_test, y_prediction)

[0 1 0 1 1]


In [28]:
print("accuracy score {:.3f}".format(acc_score))

accuracy score 0.797


In [29]:
print(confusion_matrix(y_test, y_prediction))

[[1872  333]
 [ 275  520]]


In [30]:
print(classification_report(y_test, y_prediction))

              precision    recall  f1-score   support

           0       0.87      0.85      0.86      2205
           1       0.61      0.65      0.63       795

    accuracy                           0.80      3000
   macro avg       0.74      0.75      0.75      3000
weighted avg       0.80      0.80      0.80      3000



### **Prediction Analysis**

In [31]:
#Inspecting the hyperparameters of a CART in sk-learn
print(dt.get_params())


{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': 1, 'splitter': 'best'}


In [32]:
#GridSearchCV
params_dt = { 
            'max_depth':np.arange(start=1, stop=10, step=1),
            'min_samples_leaf': np.arange(start=0.01, stop=1, step=0.10),
            'max_features': [0.2, 0.4,0.6,0.8,0.10,0.12,0.16,0.18],
            'criterion':['gini','entropy']
            }
grid_dt = GridSearchCV(estimator = dt, param_grid = params_dt, scoring = 'accuracy', cv = 10, n_jobs = -1)
grid_dt.fit(x_train, y_train)

C:\Users\HP\miniconda3\envs\ds\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.735 0.735 0.735 ...   nan   nan   nan]
  warnings.warn(


GridSearchCV(cv=10, estimator=DecisionTreeClassifier(random_state=1), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
                         'max_features': [0.2, 0.4, 0.6, 0.8, 0.1, 0.12, 0.16,
                                          0.18],
                         'min_samples_leaf': array([0.01, 0.11, 0.21, 0.31, 0.41, 0.51, 0.61, 0.71, 0.81, 0.91])},
             scoring='accuracy')

In [33]:
best_hyperparams = grid_dt.best_params_
print(best_hyperparams)

{'criterion': 'entropy', 'max_depth': 9, 'max_features': 0.8, 'min_samples_leaf': 0.01}


In [34]:
best_CV_score = grid_dt.best_score_

In [35]:
print("Accuracy of the best model: {:.3f}".format(best_CV_score))

Accuracy of the best model: 0.847


In [36]:
dt = DecisionTreeRegressor(max_depth = 9, min_samples_leaf = 0.01, random_state = 1)
dt.fit(x_train, y_train)
y_pred = dt.predict(x_test)
mse_dt = MSE(y_test, y_pred)
rmse_dt = mse_dt ** (1/2)
print("MSE of decision tree model for testing data:{:.3f}".format(mse_dt))
print("RMSE score Decison Tree Regressor model for testing data:{:.3f}".format(rmse_dt))

MSE of decision tree model for testing data:0.104
RMSE score Decison Tree Regressor model for testing data:0.323


In [37]:
from sklearn.metrics import roc_auc_score
top_model = grid_dt.best_estimator_


In [38]:
y_pred_proba = top_model.predict_proba(x_test)[:,1]


In [39]:
roc_auc = roc_auc_score(y_test, y_pred_proba)

In [40]:
print("ROC AUC Score {:.3f}".format(roc_auc))

ROC AUC Score 0.909


In [41]:
#AUC Using cross_validation 
from sklearn.model_selection import cross_val_score 
cross_val_scores = cross_val_score(dt, x_train, y_train, cv = 5, scoring = 'roc_auc')
print(cross_val_scores) #AUC scores computed using 5-fold cross validation

[0.90657402 0.89602683 0.91439909 0.90108034 0.90285984]


### **Explanation for the accuracy score and the Mean Squared Error**

The accuracy score for the original model was reported to be 0.797. However, after hyperparameter adjustment, the best model's accuracy score was increased to.847. GridSearchCV was used to select the best hyperparameters. Entropy was the best criterion, with an optimal max depth of nine, max features of 0.80, and a min sample leaf of 0.0100. The AOC score for ROU was also reported to be.909.
The MSE (Mean Squared Error), which is calculated by squaring the average difference over the data set, represents the difference between actual and projected values. It's a measure of how close a fitted line is to actual data points. RMSE is the error rate multiplied by the square root of MSE (Root Mean Squared Error).
For testing data, the MSE of the decision tree model is 0.104, while the RMSE score for the decision tree regressor model is 0.323.
The MSE and RMSE are lower the closer the fit is to the data set.
With a low mean squared error, we have a good model with high prediction accuracy.

### **Limitation of analysis**


One of the decision trees' drawbacks is that they are very unstable when compared to other choice predictors. A slight change in the data might cause a significant structural transformation of the tree structure, resulting in an outcome that differs from what consumers would expect in a typical event.

### **Recommendation**
The organization can use this decision tree algorithm to anticipate the customer's tenure based on the analysis done and the model's accuracy score. Additional parameter adjustments on additional parameters and the use of more labeled data to train the model can improve the model even further.
It's also vital for decision-makers to know that the predictor factors that were analyzed produce a reasonable accuracy score. Thus, we should look at the characteristics that are common among individuals who are leaving the organization and try to reduce the possibility that they will occur with any specific customer in the future. This implies that as a client signs up to more than just the company's services, they are less likely to depart.